In [2]:
using RCall, LightGraphs, DataFrames, Plots, Measures, CSV, StatsBase

In [3]:
default(dpi = 400)

In [4]:
# a function for BGSM_cartesian_biclust (Section 5.2.)

# y       : a matrix of data
# init    : initialization of y

function BGSM_cartesian_biclust(y;
                                ind1 = 1:size(y,1),
                                ind2 = 1:size(y,2),
                                init = y,
                                v0 = 1e-1,
                                convtol = 1e-14,  
                                iter = 100,
                                verbose = true)
                            
    
    # get size
    n1,n2 = size(y); n = n1 * n2;
    
    
    # initialize
    theta = copy(y);
    k1 = length(ind1); k2 = length(ind2);
    mu1 = copy(y[ind1,:]); mu2 = copy(y[:,ind2]);
    q1 = zeros(n1,k1); q2 = zeros(n2,k2);
    sigmasq1 = 1; sigmasq2 = 1;
    
    # loop start
    for i = 1:iter
        
        # save previous iteration
        q1_old = copy(q1);
        q2_old = copy(q2);
        
        # E-step: update g1
        d1 = reshape(sum((repmat(theta,k1,1) - kron(mu1,ones(n1))).^2,2),n1,k1);
        q1 = exp.(-d1/(2*n2*v0)); q1 = q1 ./ sum(q1,2);
        
        # E-step: update g2
        d2 = reshape(sum((repmat(theta',k2,1) - kron(mu2',ones(n2))).^2,2),n2,k2);
        q2 = exp.(-d2/(2*n1*v0)); q2 = q2 ./ sum(q2,2);
        
        # M-step: update theta
        L1 = (speye(n1) - (q1 ./ sum(q1,1)) * q1')/v0;
        L2 = (speye(n2) - (q2 ./ sum(q2,1)) * q2')/v0;
        L = kron(L2, speye(n1)) + kron(speye(n2),L1);
        theta = reshape((speye(n) + L)\y[:], n1,n2);
        mu1 = (q1 ./ sum(q1,1))' * theta;
        mu2 = theta * (q2 ./ sum(q2,1));
        
        if verbose & (rem(i,5) == 0)
            @printf "%3d-th iteration done: error = %0.2e\n" i norm(q1 - q1_old) + norm(q2 - q2_old)
        end
        
        if verbose & (norm(q1 - q1_old) + norm(q2 - q2_old) < convtol)
            @printf "the algorithm converged at %3d-th iteration: error = %0.2e\n" i norm(q1 - q1_old) + norm(q2 - q2_old)
            break;
        end
        
    end
    
    Q1    = (q1 .> 0.5); Q1    = Q1 ./ sum(Q1, 2);
    Q2    = (q2 .> 0.5); Q2    = Q2 ./ sum(Q2, 2);
    final = ((Q1 ./ sum(Q1,1)) * Q1') * theta * ((Q2 ./ sum(Q2,1)) * Q2')
    
    return Dict([
                (:theta, theta), (:q1, q1), (:q2, q2), (:mu1, mu1), (:mu2, mu2), (:final, final)
                ])
end

# a function for BGSM_cartesian (Section 5.2.)

# y       : a matrix of data
# init    : initialization of y

function BGSM_kronecker_biclust(y;
                                ind1 = 1:size(y,1), ind2 = 1:size(y,2),
                                init = y,
                                v0 = 1e-1,
                                c = 1,
                                convtol = 1e-14,  
                                iter = 100,
                                verbose = true)
                            
    
    # get size
    n1, n2 = size(y); n = n1 * n2;
    k1 = length(ind1); k2 = length(ind2);
    p1 = n1 * k1; p2 = n2 * k2;
    
    # initialize
    theta = copy(y);
    mu = y[ind1,ind2];
    q1 = zeros(n1,k1); q2 = zeros(n2,k2);
    for i = 1:k1
        q1[ind1[i],i] = 1;
    end
    for i = 1:k2
        q2[ind2[i],i] = 1;
    end
    sigmasq = 1;
    
    # loop start
    for i = 1:iter
        
        # save previous iteration
        q1_old = copy(q1);
        q2_old = copy(q2);
        
        # E-step: update g1, g2
        temp = (repeat(theta, outer = [k1, k2]) - repeat(mu, inner = [n1, n2])).^2;
        q1 = reshape(exp.(-sum(q2[:]' .* temp,2)/(2*n2*v0*c)),n1,k1); q1 = q1./sum(q1,2);
        q2 = reshape(exp.(-sum(q1[:] .* temp,1)[:]/(2*n1*v0)),n2,k2); q2 = q2./sum(q2,2);
        
        # M-step: update theta
        L1 = (speye(n1) - (q1 ./ sum(q1,1)) * q1')/v0;
        L2 = (speye(n2) - (q2 ./ sum(q2,1)) * q2')/v0;
        theta = ((speye(n1) + L1)\y)/(speye(n2) + L2);
        
        # M-step: update mu
        mu = (q1 ./ sum(q1,1))' * theta * (q2 ./ sum(q2,1));
        
        if verbose & (rem(i,5) == 0)
            @printf "%2d-th iteration done: error = %0.2e\n" i norm(q1 - q1_old) + norm(q2 - q2_old)
        end
        
        if  verbose & (norm(q1 - q1_old) + norm(q2 - q2_old) < convtol)
            @printf "the algorithm converged at %2d-th iteration: error = %0.2e\n" i norm(q1 - q1_old) + norm(q2 - q2_old)
            break;
        end
        
    end
    
    Q1    = (q1 .> 0.5); Q1    = Q1 ./ sum(Q1, 2);
    Q2    = (q2 .> 0.5); Q2    = Q2 ./ sum(Q2, 2);
    
    return Dict([
                (:theta, theta), (:q1, q1), (:q2, q2), (:mu, mu), (:final, Q1*mu*Q2')
                ])
end

BGSM_kronecker_biclust (generic function with 1 method)

In [5]:
srand(1)
t = (1:40) .+ (1:40)';
y = t + 10 * randn(40,40);

In [6]:
srand(1)
i1 = sample(1:40,10, replace = false);
i2 = sample(1:40,10, replace = false);

In [44]:
srand(1)
t = (1:40) .+ (1:40)';
y = t + 1 * randn(40,40);
out11 = BGSM_cartesian_biclust(y, v0 = 1e-1 * 8, iter = 100, convtol = 1e-8,
    ind1 = i1, ind2 = i2, verbose = true)
out12 = BGSM_kronecker_biclust(y, v0 = 1e-1 * 8, iter = 100, convtol = 1e-8,
    ind1 = i1, ind2 = i2, verbose = true)

  5-th iteration done: error = 2.78e-01
 10-th iteration done: error = 2.25e-01
 15-th iteration done: error = 1.36e-01
 20-th iteration done: error = 3.12e-01
 25-th iteration done: error = 9.67e-02
 30-th iteration done: error = 4.55e-02
 35-th iteration done: error = 1.41e-02
 40-th iteration done: error = 4.15e-03
 45-th iteration done: error = 1.21e-03
 50-th iteration done: error = 3.52e-04
 55-th iteration done: error = 1.03e-04
 60-th iteration done: error = 2.99e-05
 65-th iteration done: error = 8.72e-06
 70-th iteration done: error = 2.54e-06
 75-th iteration done: error = 7.41e-07
 80-th iteration done: error = 2.16e-07
 85-th iteration done: error = 6.29e-08
 90-th iteration done: error = 1.83e-08
the algorithm converged at  93-th iteration: error = 8.75e-09
 5-th iteration done: error = 7.14e-01
10-th iteration done: error = 1.81e-01
15-th iteration done: error = 1.27e-02
20-th iteration done: error = 7.04e-04
25-th iteration done: error = 3.91e-05
30-th iteration done: e

Dict{Symbol,Array{Float64,2}} with 5 entries:
  :mu    => [59.882 66.1339 … 48.8025 76.2955; 45.8389 51.5673 … 35.0245 62.789…
  :q2    => [4.40647e-108 4.69651e-184 … 2.10704e-22 0.0; 9.14167e-104 2.45323e…
  :theta => [6.15073 6.26356 … 42.9642 43.8203; 6.61569 6.9001 … 43.7031 44.041…
  :final => [9.43394 9.43394 … NaN NaN; 9.43394 9.43394 … NaN NaN; … ; NaN NaN …
  :q1    => [1.65784e-310 1.45994e-105 … 3.0604e-168 5.99845e-235; 8.00323e-304…

In [42]:
srand(1)
t = (1:40) .+ (1:40)';
y = t + 5 * randn(40,40);
out21 = BGSM_cartesian_biclust(y, v0 = 1e-1 * 5, iter = 100, convtol = 1e-8,
    ind1 = i1, ind2 = i2, verbose = true)
out22 = BGSM_kronecker_biclust(y, v0 = 1e-1 * 5, iter = 100, convtol = 1e-8,
    ind1 = i1, ind2 = i2, verbose = true)

  5-th iteration done: error = 1.19e-04
the algorithm converged at   7-th iteration: error = 9.21e-09
 5-th iteration done: error = 6.37e-01
10-th iteration done: error = 3.12e-01
15-th iteration done: error = 2.10e-01
20-th iteration done: error = 2.91e-04
25-th iteration done: error = 4.46e-09
the algorithm converged at 25-th iteration: error = 4.46e-09


Dict{Symbol,Array{Float64,2}} with 5 entries:
  :mu    => [68.6474 63.3224 … 47.3163 75.3065; 55.4925 53.2549 … 38.7435 66.39…
  :q2    => [0.0 5.86707e-286 … 1.13516e-45 0.0; 0.0 1.06237e-280 … 3.54182e-44…
  :theta => [5.60657 4.85156 … 43.8285 45.5914; 5.79281 5.52039 … 45.1638 45.47…
  :final => [7.20298 7.20298 … 44.4344 44.4344; 7.20298 7.20298 … 44.4344 44.43…
  :q1    => [0.0 8.20327e-228 … 6.78362e-301 0.0; 0.0 2.69749e-225 … 3.4027e-29…

In [40]:
srand(1)
t = (1:40) .+ (1:40)';
y = t + 10 * randn(40,40);
out31 = BGSM_cartesian_biclust(y, v0 = 1e-1 * 5, iter = 100, convtol = 1e-8,
    ind1 = i1, ind2 = i2, verbose = true)
out32 = BGSM_kronecker_biclust(y, v0 = 1e-1 * 3, iter = 100, convtol = 1e-8,
    ind1 = i1, ind2 = i2, verbose = true)

  5-th iteration done: error = 9.86e-02
 10-th iteration done: error = 1.49e-11
the algorithm converged at  10-th iteration: error = 1.49e-11
the algorithm converged at  4-th iteration: error = 2.59e-09


Dict{Symbol,Array{Float64,2}} with 5 entries:
  :mu    => [70.9519 61.6441 … 48.7617 74.6032; 58.1107 49.4592 … 39.0035 60.62…
  :q2    => [0.0 0.0 … 1.49493e-75 0.0; 0.0 0.0 … 4.31974e-136 0.0; … ; 4.38369…
  :theta => [8.85483 1.32518 … 42.1838 43.9532; 9.87914 3.75697 … 43.3687 43.74…
  :final => [11.3839 4.68987 … 42.6815 42.6815; 11.3839 4.68987 … 42.6815 42.68…
  :q1    => [0.0 3.10477e-313 … 0.0 0.0; 0.0 2.56909e-316 … 0.0 0.0; … ; 1.0 3.…

In [52]:
F1 = Plots.heatmap(out11[:theta], title = "Cartesian: sigma = 1 ", fillcolor = :balance, legend = false,  ticks = false)
F2 = Plots.heatmap(out12[:theta], title = "Kronecker: sigma = 1 ", fillcolor = :balance, legend = false,  ticks = false)
F3 = Plots.heatmap(out21[:theta], title = "Cartesian: sigma = 5 ", fillcolor = :balance, legend = false,  ticks = false)
F4 = Plots.heatmap(out22[:theta], title = "Kronecker: sigma = 5 ", fillcolor = :balance, legend = false,  ticks = false)
F5 = Plots.heatmap(out31[:final], title = "Cartesian: sigma = 10 ", fillcolor = :balance, legend = false,  ticks = false)
F6 = Plots.heatmap(out32[:final], title = "Kronecker: sigma = 10 ", fillcolor = :balance, legend = false, ticks = false)
F = plot(F1,F3,F5,F2,F4,F6, size = (1400,900), layout = (2,3))
Plots.savefig("figure/fig_cart_vs_kron")